# Media Campaign Data Cleanup

  Author: Joshua Guillen

___


## Background
___

## Summary
___

## Table of Contents
___

## 0. Setup / Dependencies
---

Importing Google goodness

In [18]:
!pip install google-cloud
!pip install google-cloud-storage
!pip install google-cloud-bigquery[pandas]

In [17]:
from google.colab import auth
auth.authenticate_user()

Connecting to my project's data warehouse in BigQuery.

In [3]:
from google.cloud import bigquery
client = bigquery.Client(project='sfda-capstone-project')

Importing Audience Reporting, Device Location Creative Reporting, and Inventory Viewing Data Sets.


In [20]:
QUERY = """
SELECT *
FROM sfda-capstone-project.datasets.device_location_creative_reporting
"""
results = client.query(QUERY)
device_location_creative = results.to_dataframe()


In [21]:
QUERY = """
SELECT *
FROM sfda-capstone-project.datasets.inventory_viewability_reporting
"""
results = client.query(QUERY)
inventory_viewability = results.to_dataframe()

## 1. Device Location Creative Reporting

Initial View

In [23]:
device_location_creative.head()

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Device_Model,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,None,None,None,None,None,None,None,None,<NA>,<NA>,<NA>,NaN,NaN
1,Mobile_2P_Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000974,0.000313
2,Mobile_2P_In Market_Business Services or ISPs,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000583,0.000187
3,Mobile_2P_Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000662,0.000213
4,Mobile_2P_Custom Intent/Affinity,"Chicago, Illinois",Google Chrome,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.001402,0.000450


Removing Nulls on 0th Row

In [24]:
device_location_creative = device_location_creative.drop(0).reset_index(drop=True)

In [12]:
device_location_creative.head()

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Device_Model,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,Mobile_2P_Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000974,0.000313
1,Mobile_2P_In Market_Business Services or ISPs,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000583,0.000187
2,Mobile_2P_Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000662,0.000213
3,Mobile_2P_Custom Intent/Affinity,"Chicago, Illinois",Google Chrome,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.001402,0.000450
4,Mobile_2P_Custom Intent/Affinity,"Detroit, Michigan",Google Chrome,Android 8.1,300x50,Smart Phone,BQ (All Models),bq AQUARIS X,1,0,0,0.000703,0.000226


In [13]:
device_location_creative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464433 entries, 0 to 464432
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Strategy           464433 non-null  object 
 1   DMA_Name           464433 non-null  object 
 2   Browser            464433 non-null  object 
 3   Operating_System   464433 non-null  object 
 4   Creative_Size      464433 non-null  object 
 5   Device_Type        464433 non-null  object 
 6   Device_Make        464433 non-null  object 
 7   Device_Model       464433 non-null  object 
 8   Impressions        464433 non-null  Int64  
 9   Clicks             464433 non-null  Int64  
 10  Total_Conversions  464433 non-null  Int64  
 11  Gross_Cost         464433 non-null  float64
 12  Net_Cost           464433 non-null  float64
dtypes: Int64(3), float64(2), object(8)
memory usage: 47.4+ MB


## 2. Inventory Viewability Reporting

Initial View

In [25]:
inventory_viewability.head()

,Brand_Company_Name,Strategy__,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions
0,Choose your advertiser name,Desktop_2P_Affinity Categories,chicagotribune.com,Taboola,36,0,0,0.114440,0.036735,8,33
1,Choose your advertiser name,Desktop_2P_Affinity Categories,jamaicaobserver.com,Taboola,52,0,0,0.053679,0.017231,25,52
2,Choose your advertiser name,Desktop_2P_Affinity Categories,mlive.com,Taboola,3288,0,1,4.330890,1.390216,2095,2931
3,Choose your advertiser name,Desktop_2P_Affinity Categories,silive.com,Taboola,32,0,0,0.065234,0.020940,16,27
4,Choose your advertiser name,Desktop_2P_Affinity Categories,univision.com,Taboola,133,0,0,0.376884,0.120980,89,134


Data has over a million rows (1,037,468) across 11 rows!

In [28]:
inventory_viewability.shape

(1037468, 11)

Inventory Viewability dataset has financial information on cost

In [27]:
inventory_viewability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037468 entries, 0 to 1037467
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Brand_Company_Name      1037466 non-null  object 
 1   Strategy__              1037466 non-null  object 
 2   App_URL                 1037466 non-null  object 
 3   Exchange                1037466 non-null  object 
 4   Impressions             1037466 non-null  Int64  
 5   Clicks                  1037466 non-null  Int64  
 6   Total_Conversions       1037466 non-null  Int64  
 7   Gross_Cost              1037466 non-null  float64
 8   Net_Cost                1037466 non-null  float64
 9   Viewable_Impressions    1037466 non-null  Int64  
 10  Measurable_Impressions  1037466 non-null  Int64  
dtypes: Int64(5), float64(2), object(4)
memory usage: 92.0+ MB
